# Estratégia 1: Creación de Producto Artificial
En este Notebook Vamos a Generar un nuevo producto con venta 3 veces superior al producto Estrella con la intención de evitar la caída en la predicción del Estrella.

## Importamos librerías

In [2]:
###### Feature Engineering ######
import pandas as pd
import numpy as np
from window_ops.rolling import rolling_mean, rolling_max, rolling_min

###### Load the API  ######
import requests
import gzip
import io

###### Forecasting ######
from mlforecast import MLForecast
from empresa4.core import calculate_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
import lightgbm as lgb 


from empresa4.datasets import nombres_datasets, get_dataset

###### Visualize ######
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Set the display.max_columns to None to display all columns
pd.set_option('display.max_columns', None)

## Funciones

In [9]:
def cargar_dataset(url, sep='\t', header='infer'):
    """
    Lee un archivo CSV comprimido con gzip desde una URL y lo carga en un DataFrame de Pandas.

    :param url: String con la URL del archivo .gz.
    :param sep: String delimitador para separar campos en el archivo CSV.
    :param header: Parámetro para determinar la fila que debe usarse como nombre de las columnas.
    :return: DataFrame de Pandas con los datos del archivo CSV.
    """
    # Realizar la petición HTTP para obtener el contenido del archivo
    response = requests.get(url)

    # Asegurarse de que la petición fue exitosa
    if response.status_code == 200:
        # Descomprimir el contenido en memoria y leer directamente en un DataFrame
        with io.BytesIO(response.content) as bytes_io:
            with gzip.open(bytes_io, 'rt') as read_file:
                df = pd.read_csv(read_file, sep=sep, header=header)
        return df
    else:
        raise IOError(f"Error al descargar el archivo: {response.status_code}")
    
def read_csv_from_url(url, sep='\t', header='infer'):
    response = requests.get(url)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep=sep, header=header)
        return df
    else:
        raise IOError(f"Error al descargar el archivo: {response.status_code}")
    
def plotear_grafico(df):
    # Convertir el 'periodo' a formato de fecha para mejor visualización en el gráfico
    df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

    # Crear nuevamente el gráfico de línea con Seaborn
    plt.figure(figsize=(14, 7))
    line_plot = sns.lineplot(data=df, x='periodo', y='tn', marker='o', color='blue', linewidth=2.5)

    # Títulos y etiquetas
    line_plot.set_title('Evolución de Toneladas a lo largo del Tiempo', fontsize=16)
    line_plot.set_xlabel('Periodo', fontsize=14)
    line_plot.set_ylabel('Toneladas (tn)', fontsize=14)

    # Establecer el formato del eje x para mostrar todos los periodos
    line_plot.xaxis.set_major_locator(plt.MaxNLocator(len(df['periodo'])))
    line_plot.set_xticklabels(df['periodo'].dt.strftime('%Y-%m'), rotation=45, ha='right')

    # Mostrar el gráfico
    plt.tight_layout()
    plt.show()
    
def fill_missing_months_multi_product(df):
    # Find the earliest and latest period (assuming the format is YYYYMM)
    min_year = df['periodo'].min() // 100
    max_year = df['periodo'].max() // 100

    # Generate all combinations of year and months
    all_periods = pd.DataFrame({
        'periodo': [year * 100 + month for year in range(min_year, max_year + 1) for month in range(1, 13)]
    })

    product_ids = df['product_id'].unique()
    all_period_product_combinations = (
        all_periods.assign(key=1)
        .merge(pd.DataFrame({'product_id': product_ids, 'key': 1}), on='key')
        .drop(columns=['key'])  # Using the 'columns' parameter for clarity
    )

    # Merge with the original DataFrame
    df_complete = all_period_product_combinations.merge(df, on=['periodo', 'product_id'], how='left')

    # Fill the 'tn' column with zeros where there are NaNs
    df_complete['tn'] = df_complete['tn'].fillna(0)

    # Forward fill or backfill other columns that don't change for each product
    columns_to_fill = ['plan_precios_cuidados', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']
    df_complete.sort_values(by=['product_id', 'periodo'], inplace=True)
    df_complete[columns_to_fill] = df_complete.groupby('product_id')[columns_to_fill].ffill().bfill()
    return df_complete

def generar_delta_lags_optimizado(df, prefijo_lag='lag_', prefijo_delta_lag='delta_lag_'):
    """
    Genera delta lags de manera más eficiente para una columna específica de un DataFrame de pandas.

    :param df: DataFrame original que ya contiene las columnas de lags.
    :param prefijo_lag: Prefijo utilizado para las columnas de lags en el DataFrame.
    :param prefijo_delta_lag: Prefijo utilizado para las nuevas columnas de delta lags.
    :return: DataFrame con las columnas de delta lags añadidas.
    """
    
    # Identificar todas las columnas de lags en el DataFrame
    columnas_lags = [col for col in df.columns if col.startswith(prefijo_lag)]
    
    # Generar las columnas de delta lags
    for i in range(len(columnas_lags) - 1):
        col_lag_actual = columnas_lags[i]
        col_lag_siguiente = columnas_lags[i + 1]
        df[prefijo_delta_lag + str(i + 1)] = df[col_lag_actual] - df[col_lag_siguiente]
    
    return df

# Re-creating the optimized lag generation function after the reset
def generar_lags_optimizado(df, nombre_col_original='tn', cant_lags=12):
    """
    Genera lags de manera más eficiente para una columna específica de un DataFrame de pandas.

    :param df: DataFrame original.
    :param nombre_col_original: Nombre de la columna sobre la cual se quieren calcular los lags.
    :param cant_lags: Cantidad de lags a generar.
    :return: DataFrame con las columnas de lags añadidas.
    """
    
    # Obtener una lista de productos únicos
    productos_unicos = df['product_id'].unique()
    
    # Preparar un contenedor para los DataFrames de lags
    lags_list = []
    
    # Iterar sobre cada producto único
    for producto in productos_unicos:
        # Filtrar el DataFrame por producto
        df_producto = df[df['product_id'] == producto].copy()
        
        # Generar lags para la columna deseada
        for lag in range(1, cant_lags + 1):
            df_producto[f'lag_{lag}'] = df_producto[nombre_col_original].shift(lag)
        
        # Añadir el DataFrame de lags a la lista
        lags_list.append(df_producto)
    
    # Concatenar todos los DataFrames de lags
    df_con_lags = pd.concat(lags_list).sort_index()
    
    return df_con_lags

def generar_medias_optimizado(df, nombre_col_original='tn', lista_medias_target=[2,3,4,6,10], prefijo_col_media='media'):
    """
    Genera las medias móviles de los últimos N meses de manera más eficiente para una columna específica de un DataFrame de pandas.

    :param df: DataFrame original.
    :param nombre_col_original: Nombre de la columna sobre la cual se quieren calcular las medias móviles.
    :param lista_medias_target: Lista de enteros que representan el número de meses para calcular la media móvil.
    :param prefijo_col_media: Prefijo para las nuevas columnas de medias móviles.
    :return: DataFrame con las columnas de medias móviles añadidas.
    """
    
    # Obtener una lista de productos únicos
    productos_unicos = df['product_id'].unique()
    
    # Preparar un contenedor para los DataFrames de medias móviles
    medias_list = []
    
    # Iterar sobre cada producto único
    for producto in productos_unicos:
        # Filtrar el DataFrame por producto
        df_producto = df[df['product_id'] == producto].copy()
        
        # Generar medias móviles para cada ventana especificada en lista_medias_target
        for ventana in lista_medias_target:
            col_media = f'{nombre_col_original}_{prefijo_col_media}_{ventana}'
            df_producto[col_media] = df_producto[nombre_col_original].rolling(window=ventana).mean().shift()
        
        # Añadir el DataFrame de medias móviles a la lista
        medias_list.append(df_producto)
    
    # Concatenar todos los DataFrames de medias móviles
    df_con_medias = pd.concat(medias_list).sort_index()
    
    return df_con_medias

## Importamos Datasets

In [10]:
# URL del archivo .gz
url_tb_sellout_02 = 'https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_sellout_02.txt.gz'
url_tb_stocks_02 = 'https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_stocks_02.txt.gz'
url_tb_productos_02 = 'https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_productos_02.txt'

# Uso de la función
df_tb_sellout_02 = cargar_dataset(url_tb_sellout_02)
df_tb_stocks_02 = cargar_dataset(url_tb_stocks_02)
df_tb_productos_02= read_csv_from_url(url_tb_productos_02)

## Pre Processing

In [12]:
# Juntamos los datasets Sellout con el maestro de productos
df_sellout_with_cat = pd.merge(df_tb_sellout_02, df_tb_productos_02[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']], on='product_id', how='left')
# Agrupamos por categorías de productos
df_aggregated = df_sellout_with_cat.groupby(['periodo', 'product_id', 'plan_precios_cuidados', 
                                             'cat1', 'cat2', 'cat3', 'brand', 'sku_size']).agg({'cust_request_tn': 'sum','tn': 'sum'}).reset_index()

## Feature Engineering

In [123]:
data_con_venenosa = df_aggregated.copy()

# Filtrar el DataFrame para el periodo 201903
filtro = data_con_venenosa['periodo'] == 201904

# Multiplicar la columna 'tn' por 100.000.000 solo para ese periodo
data_con_venenosa.loc[filtro, 'tn'] *= 100000000

In [125]:
data_con_venenosa[data_con_venenosa.product_id == 20001]

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn
0,201701,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,937.72717,9.347722e+02
759,201702,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,833.72187,7.980162e+02
1520,201703,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1330.74697,1.303358e+03
2290,201704,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1132.94430,1.069961e+03
3063,201705,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1550.68936,1.502201e+03
3853,201706,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1575.82891,1.520065e+03
4657,201707,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1086.47101,1.030674e+03
5475,201708,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1289.66869,1.267395e+03
6301,201709,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1356.96103,1.316946e+03
7125,201710,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1441.60247,1.439756e+03


In [126]:
# Llamamos a la función para rellenar los registros faltantes
df_complete = fill_missing_months_multi_product(data_con_venenosa)

In [127]:
df_complete[df_complete.product_id == 20001]

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn
0,201701,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,937.72717,9.347722e+02
1189,201702,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,833.72187,7.980162e+02
2378,201703,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1330.74697,1.303358e+03
3567,201704,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1132.94430,1.069961e+03
4756,201705,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1550.68936,1.502201e+03
5945,201706,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1575.82891,1.520065e+03
7134,201707,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1086.47101,1.030674e+03
8323,201708,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1289.66869,1.267395e+03
9512,201709,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1356.96103,1.316946e+03
10701,201710,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1441.60247,1.439756e+03


### Creamos un nuevo producto artificial con venta 3 veces la del producto Estrella

In [128]:
# Crear un nuevo DataFrame con los mismos periodos que df_complete
periodos = df_complete['periodo'].unique()  # Extraer los periodos únicos
datos_ficticios = {
    'periodo': periodos,
    'product_id': 99999,  # ID del producto ficticio
    'plan_precios_cuidados': 0.0,
    'cat1': 'HC',
    'cat2': 'Artificial',
    'cat3': 'Artificial',
    'brand': 'Producto_artificial',
    'sku_size': 3000.0,
    # Inicializamos cust_request_tn y tn con ceros, los actualizaremos luego
    'cust_request_tn': [0] * len(periodos),
    'tn': [0] * len(periodos)
}

df_ficticio = pd.DataFrame(datos_ficticios)

# Ahora, actualizamos 'cust_request_tn' y 'tn' con 10 veces los valores del producto 20001
# Asumimos que 'product_id' es único por periodo en df_complete
for periodo in periodos:
    # Encuentra los valores para el producto 20001 en el periodo actual
    valores_producto = df_complete[(df_complete['periodo'] == periodo) & (df_complete['product_id'] == 20001)]
    if not valores_producto.empty:
        df_ficticio.loc[df_ficticio['periodo'] == periodo, 'cust_request_tn'] = valores_producto['cust_request_tn'].values[0] * 3
        df_ficticio.loc[df_ficticio['periodo'] == periodo, 'tn'] = valores_producto['tn'].values[0] * 3

# Concatenar el DataFrame ficticio con el original
df_complete_con_ficticio = pd.concat([df_complete, df_ficticio], ignore_index=True)


C:\Users\Lean\AppData\Local\Temp\ipykernel_22048\2823364550.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2813.18151' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_ficticio.loc[df_ficticio['periodo'] == periodo, 'cust_request_tn'] = valores_producto['cust_request_tn'].values[0] * 3
C:\Users\Lean\AppData\Local\Temp\ipykernel_22048\2823364550.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2804.31666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_ficticio.loc[df_ficticio['periodo'] == periodo, 'tn'] = valores_producto['tn'].values[0] * 3


In [129]:
df_complete_con_ficticio[df_complete_con_ficticio.product_id == 20001]

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn
0,201701,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,937.72717,9.347722e+02
1,201702,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,833.72187,7.980162e+02
2,201703,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1330.74697,1.303358e+03
3,201704,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1132.94430,1.069961e+03
4,201705,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1550.68936,1.502201e+03
5,201706,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1575.82891,1.520065e+03
6,201707,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1086.47101,1.030674e+03
7,201708,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1289.66869,1.267395e+03
8,201709,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1356.96103,1.316946e+03
9,201710,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1441.60247,1.439756e+03


In [130]:
# Generamos los lags
df_complete_lags = generar_lags_optimizado(df_complete_con_ficticio)

In [131]:
df_complete_lags[df_complete_lags.product_id == 20001]

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,201701,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,937.72717,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201702,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,833.72187,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201703,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1330.74697,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201704,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1132.94430,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201705,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1550.68936,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,201706,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1575.82891,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,201707,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1086.47101,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN
7,201708,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1289.66869,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN
8,201709,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1356.96103,1.316946e+03,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN
9,201710,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1441.60247,1.439756e+03,1.316946e+03,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,934.77222,NaN,NaN,NaN


In [132]:
# Utilizamos el DataFrame con lags generado anteriormente para crear los delta lags
df_complete_delta_lags = generar_delta_lags_optimizado(df_complete_lags)

In [133]:
df_complete_delta_lags[df_complete_delta_lags.product_id == 20001]

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,delta_lag_1,delta_lag_2,delta_lag_3,delta_lag_4,delta_lag_5,delta_lag_6,delta_lag_7,delta_lag_8,delta_lag_9,delta_lag_10,delta_lag_11
0,201701,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,937.72717,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201702,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,833.72187,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201703,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1330.74697,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201704,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1132.94430,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201705,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1550.68936,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,201706,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1575.82891,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,201707,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1086.47101,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.786407e+01,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN
7,201708,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1289.66869,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,-4.893915e+02,1.786407e+01,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN
8,201709,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1356.96103,1.316946e+03,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,2.367207e+02,-4.893915e+02,1.786407e+01,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN
9,201710,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1441.60247,1.439756e+03,1.316946e+03,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,934.77222,NaN,NaN,NaN,4.955142e+01,2.367207e+02,-4.893915e+02,1.786407e+01,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN


In [134]:
# Utilizamos el DataFrame con lags generado anteriormente para crear las medias móviles
df_final = generar_medias_optimizado(df_complete_delta_lags)

In [135]:
df_final[df_final.product_id == 20001]

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,delta_lag_1,delta_lag_2,delta_lag_3,delta_lag_4,delta_lag_5,delta_lag_6,delta_lag_7,delta_lag_8,delta_lag_9,delta_lag_10,delta_lag_11,tn_media_2,tn_media_3,tn_media_4,tn_media_6,tn_media_10
0,201701,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,937.72717,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201702,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,833.72187,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201703,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1330.74697,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.663942e+02,NaN,NaN,NaN,NaN
3,201704,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1132.94430,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.050687e+03,1.012049e+03,NaN,NaN,NaN
4,201705,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1550.68936,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.186660e+03,1.057112e+03,1.026527e+03,NaN,NaN
5,201706,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1575.82891,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.286081e+03,1.291840e+03,1.168384e+03,NaN,NaN
6,201707,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1086.47101,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.786407e+01,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.511133e+03,1.364076e+03,1.348896e+03,1.188062e+03,NaN
7,201708,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1289.66869,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,NaN,-4.893915e+02,1.786407e+01,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,NaN,1.275370e+03,1.350980e+03,1.280725e+03,1.204046e+03,NaN
8,201709,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1356.96103,1.316946e+03,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,9.347722e+02,NaN,NaN,NaN,NaN,2.367207e+02,-4.893915e+02,1.786407e+01,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,NaN,1.149034e+03,1.272711e+03,1.330084e+03,1.282276e+03,NaN
9,201710,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1441.60247,1.439756e+03,1.316946e+03,1.267395e+03,1.030674e+03,1.520065e+03,1.502201e+03,1.069961e+03,1.303358e+03,7.980162e+02,934.77222,NaN,NaN,NaN,4.955142e+01,2.367207e+02,-4.893915e+02,1.786407e+01,4.322400e+02,-2.333964e+02,5.053415e+02,-1.367560e+02,NaN,NaN,NaN,1.292170e+03,1.205005e+03,1.283770e+03,1.284540e+03,NaN


## Cross Validation por corte temporal

In [140]:
# Assuming df_final is the DataFrame you want to split into train and validation sets:
df_final['periodo'] = pd.to_datetime(df_final['periodo'], format='%Y%m')

# Cortamos hasta Febrero inclusive como TRAIN
train = df_final[df_final['periodo'] <= pd.to_datetime('201902', format='%Y%m')]
# Usamos MARZO como test
valid_marzo = df_final[df_final['periodo'] == pd.to_datetime('201903', format='%Y%m')]

# Para predecir Marzo
X_train = train.drop(columns=['tn', 'periodo'])
y_train = train['tn']
X_test = valid_marzo.drop(columns=['tn', 'periodo'])
y_test = valid_marzo['tn']

## Modeling

### LGBM

In [141]:
# Create the LightGBM dataset
categorical_features = ['cat1', 'cat2', 'cat3', 'brand']

# Convert categorical features to 'category' type

for feature in categorical_features:
    X_train[feature] = X_train[feature].astype('category')
    X_test[feature] = X_test[feature].astype('category')


train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features, free_raw_data=False)


# Define the parameters
params = {
    'objective': 'regression',
    'metric': 'mae',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}

# Train the model
gbm = lgb.train(params, train_data, num_boost_round=100)

In [142]:
# Predict on the test set
y_pred_marzo = gbm.predict(X_test)

In [144]:
y_test_array = y_test.to_numpy()
error = calculate_error(y_pred_marzo, y_test_array)

# Print the error for todos los productos
print(f'The error of the forecast is: {error}')

The error of the forecast is: 0.046477884142888


In [145]:
# Suponiendo que 'valid_marzo' es tu DataFrame y 'y_pred' es la serie de predicciones:
valid_marzo['y_pred_marzo'] = y_pred_marzo
producto_estrella_marzo = valid_marzo[valid_marzo.product_id ==20001]

error = calculate_error(list(producto_estrella_marzo.y_pred_marzo), list(producto_estrella_marzo.tn))

# Print the error for todos los productos
print(f'The error of the forecast is: {error}')

The error of the forecast is: 0.09220885150933848


C:\Users\Lean\AppData\Local\Temp\ipykernel_22048\2003783269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_marzo['y_pred_marzo'] = y_pred_marzo


#### Incluimos las prediccioines de MARZO en el TRAIN para predecir ABRIL

In [158]:
# Actualizar el conjunto de entrenamiento
new_row = X_test.copy()
train_con_marzo = train.copy()

new_row['tn'] = y_pred_marzo  # Añade la predicción como el valor real de 'tn'
new_row['periodo'] = '2019-03-01'  # Añadir la columna 'periodo'

train_con_marzo = pd.concat([train_con_marzo, new_row]) # contiene los datos reales hasta FEB19 y agrega la prediccion de MAR19

valid_abril = df_final[['product_id','periodo','plan_precios_cuidados','cat1','cat2','cat3','brand','sku_size','cust_request_tn','tn']].copy()

In [163]:
valid_abril = valid_abril[valid_abril.periodo == '2019-04-01']
valid_abril

,product_id,periodo,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn
27,20001,2019-04-01,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1757.73271,1.647638e+11
63,20002,2019-04-01,0.0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,1360.44402,1.287623e+11
99,20003,2019-04-01,0.0,FOODS,ADEREZOS,Mayonesa,NATURA,475.0,569.69482,5.653377e+10
135,20004,2019-04-01,0.0,FOODS,ADEREZOS,Mayonesa,NATURA,240.0,468.21007,4.667090e+10
171,20005,2019-04-01,0.0,FOODS,ADEREZOS,Mayonesa,NATURA,120.0,629.64621,6.249988e+10
...,...,...,...,...,...,...,...,...,...,...
42697,21276,2019-04-01,0.0,PC,PIEL1,Cara,NIVEA,140.0,0.10173,1.017300e+07
42733,21282,2019-04-01,0.0,PC,CABELLO,POST WASH,SHAMPOO1,150.0,NaN,0.000000e+00
42769,21287,2019-04-01,0.0,FOODS,SOPAS Y CALDOS,Sazonadores,MAGGI,10.0,NaN,0.000000e+00
42805,21294,2019-04-01,0.0,PC,CABELLO,Tratamiento Fem,NIVEA,350.0,NaN,0.000000e+00


In [181]:
df_complete_2 = pd.concat([train_con_marzo, valid_abril])
# Convertir la columna 'periodo' a datetime
df_complete_2['periodo'] = pd.to_datetime(df_complete_2['periodo']).dt.date
df_filtrado = df_complete_2[df_complete_2['periodo'] == pd.to_datetime('2019-04-01').date()]
df_complete_2.tail()

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,delta_lag_1,delta_lag_2,delta_lag_3,delta_lag_4,delta_lag_5,delta_lag_6,delta_lag_7,delta_lag_8,delta_lag_9,delta_lag_10,delta_lag_11,tn_media_2,tn_media_3,tn_media_4,tn_media_6,tn_media_10
42697,2019-04-01,21276,0.0,PC,PIEL1,Cara,NIVEA,140.0,0.10173,1.017300e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,2019-04-01,21282,0.0,PC,CABELLO,POST WASH,SHAMPOO1,150.0,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42769,2019-04-01,21287,0.0,FOODS,SOPAS Y CALDOS,Sazonadores,MAGGI,10.0,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42805,2019-04-01,21294,0.0,PC,CABELLO,Tratamiento Fem,NIVEA,350.0,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42841,2019-04-01,99999,0.0,HC,Artificial,Artificial,Producto_artificial,3000.0,5273.19813,4.942915e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
df_filtrado

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,delta_lag_1,delta_lag_2,delta_lag_3,delta_lag_4,delta_lag_5,delta_lag_6,delta_lag_7,delta_lag_8,delta_lag_9,delta_lag_10,delta_lag_11,tn_media_2,tn_media_3,tn_media_4,tn_media_6,tn_media_10
27,2019-04-01,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1757.73271,1.647638e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,2019-04-01,20002,0.0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,1360.44402,1.287623e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,2019-04-01,20003,0.0,FOODS,ADEREZOS,Mayonesa,NATURA,475.0,569.69482,5.653377e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,2019-04-01,20004,0.0,FOODS,ADEREZOS,Mayonesa,NATURA,240.0,468.21007,4.667090e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,2019-04-01,20005,0.0,FOODS,ADEREZOS,Mayonesa,NATURA,120.0,629.64621,6.249988e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42697,2019-04-01,21276,0.0,PC,PIEL1,Cara,NIVEA,140.0,0.10173,1.017300e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,2019-04-01,21282,0.0,PC,CABELLO,POST WASH,SHAMPOO1,150.0,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42769,2019-04-01,21287,0.0,FOODS,SOPAS Y CALDOS,Sazonadores,MAGGI,10.0,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42805,2019-04-01,21294,0.0,PC,CABELLO,Tratamiento Fem,NIVEA,350.0,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Generamos el Feature Engineering necesario

In [184]:
# Generamos los lags
df_complete_lags_2 = generar_lags_optimizado(df_complete_2)
# Utilizamos el DataFrame con lags generado anteriormente para crear los delta lags
df_complete_delta_lags_2 = generar_delta_lags_optimizado(df_complete_lags_2)
# Utilizamos el DataFrame con lags generado anteriormente para crear las medias móviles
df_final_2 = generar_medias_optimizado(df_complete_delta_lags_2)

In [185]:
probando = df_final_2[(df_final_2.product_id==20001)].tail(3)
probando

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,delta_lag_1,delta_lag_2,delta_lag_3,delta_lag_4,delta_lag_5,delta_lag_6,delta_lag_7,delta_lag_8,delta_lag_9,delta_lag_10,delta_lag_11,tn_media_2,tn_media_3,tn_media_4,tn_media_6,tn_media_10
25,2019-02-01,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1343.99435,1.259094e+03,1275.77351,1486.68669,1813.01511,2295.19832,1438.67455,1800.96168,1470.41009,1150.79169,1293.89788,1251.28462,1856.83534,1043.76470,-210.91318,-326.32842,-482.18321,856.52377,-362.28713,330.55159,319.61840,-143.10619,42.61326,-605.55072,813.07064,1381.230100,1525.158437,1717.668407,1685.051643,1527.669414
26,2019-03-01,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1607.18772,1.606264e+03,1259.09363,1275.77351,1486.68669,1813.01511,2295.19832,1438.67455,1800.96168,1470.41009,1150.79169,1293.89788,1251.28462,1856.83534,-16.67988,-210.91318,-326.32842,-482.18321,856.52377,-362.28713,330.55159,319.61840,-143.10619,42.61326,-605.55072,1267.433570,1340.517943,1458.642235,1594.740302,1528.450315
27,2019-04-01,20001,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1757.73271,1.647638e+11,1606.26408,1259.09363,1275.77351,1486.68669,1813.01511,2295.19832,1438.67455,1800.96168,1470.41009,1150.79169,1293.89788,1251.28462,347.17045,-16.67988,-210.91318,-326.32842,-482.18321,856.52377,-362.28713,330.55159,319.61840,-143.10619,42.61326,1432.678855,1380.377073,1406.954477,1622.671890,1559.686935


In [190]:
# Separamos la Data entre TRAIN y TEST
df_final_2['periodo'] = pd.to_datetime(df_final_2['periodo'], format='%Y-%m-%d')

train_abril = df_final_2[df_final_2['periodo'] <= pd.to_datetime('2019-03-01')] # Vuelvo a hacer el corte hasta MArzo con MAR19 predicho
valid_abril = df_final_2[df_final_2['periodo'] == pd.to_datetime('2019-04-01')] # vuelvo a asignar ABRIL pero esta vez con los LAGS y DELTAS

# Cross Validation por corte temporal
X_train_abril = train_abril.drop(['tn', 'periodo'], axis=1)
y_train_abril = train_abril['tn']

X_test_abril = valid_abril.drop(columns=['tn', 'periodo'])
y_test_abril = valid_abril['tn']

#### Modelamos nuevamente

In [191]:
# Create the LightGBM dataset
categorical_features = ['cat1', 'cat2', 'cat3', 'brand']

# Convert categorical features to 'category' type

for feature in categorical_features:
    X_train_abril[feature] = X_train_abril[feature].astype('category')
    X_test_abril[feature] = X_test_abril[feature].astype('category')


train_data_abril = lgb.Dataset(X_train_abril, label=y_train_abril, categorical_feature=categorical_features, free_raw_data=False)


# Define the parameters
params = {
    'objective': 'regression',
    'metric': 'mae',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}

# Train the model
gbm = lgb.train(params, train_data_abril, num_boost_round=100)

In [192]:
# Predict on the test set
y_pred_abril = gbm.predict(X_test_abril)

In [193]:
y_test_array = y_test_abril.to_numpy()
error = calculate_error(y_pred_abril, y_test_array)

# Print the error
print(f'The error of the forecast is: {error}')

The error of the forecast is: 0.9999999901377329


In [194]:
# Suponiendo que 'valid_marzo' es tu DataFrame y 'y_pred' es la serie de predicciones:
valid_abril['y_pred_abril'] = y_pred_abril
producto_estrella_abril = valid_abril[valid_abril.product_id ==20001]

error = calculate_error(list(producto_estrella_abril.y_pred_abril), list(producto_estrella_abril.tn))

# Print the error for todos los productos
print(f'The error of the forecast is: {error}')

The error of the forecast is: 0.9999999894993347


C:\Users\Lean\AppData\Local\Temp\ipykernel_22048\1992873493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_abril['y_pred_abril'] = y_pred_abril


In [195]:
# Inicializa la columna de error con NaNs o ceros si prefieres
valid_abril['error'] = float('nan')

unique_product_ids = valid_abril['product_id'].unique()

for product_id in unique_product_ids:
    producto_estrella_abril = valid_abril[valid_abril['product_id'] == product_id]
    
    # Asegúrate de que la suma de los valores reales no sea cero
    if producto_estrella_abril['tn'].sum() != 0:
        error = calculate_error(list(producto_estrella_abril['y_pred_abril']), list(producto_estrella_abril['tn']))
        # Asigna el error calculado a las filas correspondientes en el DataFrame original
        valid_abril.loc[valid_abril['product_id'] == product_id, 'error'] = error
    else:
        # Asigna un valor de error como NaN o algún indicador de error no calculable
        valid_abril.loc[valid_abril['product_id'] == product_id, 'error'] = float('nan')

C:\Users\Lean\AppData\Local\Temp\ipykernel_22048\2867215060.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_abril['error'] = float('nan')


In [196]:
valid_abril.sort_values('error', ascending= False).head(10)

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,delta_lag_1,delta_lag_2,delta_lag_3,delta_lag_4,delta_lag_5,delta_lag_6,delta_lag_7,delta_lag_8,delta_lag_9,delta_lag_10,delta_lag_11,tn_media_2,tn_media_3,tn_media_4,tn_media_6,tn_media_10,y_pred_abril,error
711,2019-04-01,20020,0.0,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,636.05070,6.240999e+10,655.272352,421.49450,548.86536,374.73944,380.01602,332.56204,359.71352,471.32474,634.23362,697.97146,918.39824,756.43272,233.777852,-127.37086,174.12592,-5.27658,47.45398,-27.15148,-111.61122,-162.90888,-63.73784,-220.42678,161.96552,538.383426,541.877404,500.092913,452.158285,487.619305,520.286026,1.0
34597,2019-04-01,20985,0.0,PC,DENTAL,CREMA,COLGATE,90.0,0.38631,3.863100e+07,0.889944,0.70769,0.37155,0.69590,0.33615,0.51601,0.61921,0.80495,0.52195,1.05853,0.84626,0.57201,0.182254,0.33614,-0.32435,0.35975,-0.17986,-0.10320,-0.18574,0.28300,-0.53658,0.21227,0.27425,0.798817,0.656395,0.666271,0.586207,0.652188,0.328326,1.0
36937,2019-04-01,21057,0.0,PC,PIEL1,Cara,LANCOME,50.0,0.38541,3.854100e+07,0.739640,0.23477,0.12009,0.35268,0.46951,1.08870,0.01201,0.49247,0.72724,0.83642,0.65188,1.26342,0.504870,0.11468,-0.23259,-0.11683,-0.61919,1.07669,-0.48046,-0.23477,-0.10918,0.18454,-0.61154,0.487205,0.364833,0.361795,0.500898,0.507353,0.328326,1.0
42841,2019-04-01,99999,0.0,HC,Artificial,Artificial,Producto_artificial,3000.0,5273.19813,4.942915e+11,4075.669179,3777.28089,3827.32053,4460.06007,5439.04533,6885.59496,4316.02365,5402.88504,4411.23027,3452.37507,3881.69364,3753.85386,298.388289,-50.03964,-632.73954,-978.98526,-1446.54963,2569.57131,-1086.86139,991.65477,958.85520,-429.31857,127.83978,3926.475035,3893.423533,4035.082667,4744.161827,4604.748499,4214.225893,1.0
22825,2019-04-01,20637,0.0,PC,DEOS,Stick,NIVEA,50.0,0.37456,3.745600e+07,3.738885,4.66338,4.00873,4.92167,4.28775,4.73217,4.81740,3.98088,4.78519,3.26566,5.63963,6.40351,-0.924495,0.65465,-0.91294,0.63392,-0.44442,-0.08523,0.83652,-0.80431,1.51953,-2.37397,-0.76388,4.201133,4.136998,4.333166,4.392098,4.320172,0.328326,1.0
31789,2019-04-01,20901,0.0,PC,DEOS,Aero,NIVEA,55.0,1.36285,1.362260e+08,1.357785,1.24372,0.31082,1.30442,1.14051,2.00258,1.34453,1.72904,1.63757,1.43474,2.24668,1.18709,0.114065,0.93290,-0.99360,0.16391,-0.86207,0.65805,-0.38451,0.09147,0.20283,-0.81194,1.05959,1.300753,0.970775,1.054186,1.226639,1.350572,1.211952,1.0
63,2019-04-01,20002,0.0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,1360.44402,1.287623e+11,1132.192992,1043.01349,1266.78751,1009.45458,1766.81068,1378.49032,954.23575,1161.88430,977.40239,1033.82845,1103.39191,999.20934,89.179502,-223.77402,257.33293,-757.35610,388.32036,424.25457,-207.64855,184.48191,-56.42606,-69.56346,104.18257,1087.603241,1147.331331,1112.862143,1266.124929,1172.410046,1147.036955,1.0
207,2019-04-01,20006,0.0,HC,VAJILLA,Cristalino,LIMPIEX,750.0,909.71571,8.354788e+10,567.454271,479.99914,578.74461,407.75925,566.66809,513.15472,478.04388,615.70617,515.20419,468.15260,865.28861,748.44391,87.455131,-98.74547,170.98536,-158.90884,53.51337,35.11084,-137.66229,100.50198,47.05159,-397.13601,116.84470,523.726705,542.066007,508.489318,518.963347,519.088692,749.204737,1.0
30961,2019-04-01,20874,0.0,PC,PIEL2,Liquido,NIVEA,250.0,1.34767,1.347670e+08,2.712151,1.62945,1.30864,1.25558,1.69077,1.39513,1.56252,1.76896,1.52072,2.23766,1.30589,2.39672,1.082701,0.32081,0.05306,-0.43519,0.29564,-0.16739,-0.20644,0.24824,-0.71694,0.93177,-1.09083,2.170801,1.883414,1.726455,1.665287,1.708158,1.211952,1.0
42517,2019-04-01,21265,0.0,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32.0,0.36405,3.640500e+07,0.132702,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.132702,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.066351,0.044234,0.033176,0.022117,0.013270,0.328326,1.0
